# Proyecto Final Arquitectura de Computadores: "Construyendo un compilador"

Este Proyecto esta basado en una implementación de los proyectos # 10 y 11 del cursos "Nand2Tetris", que cual se basa en una implementación de un compilador para un lenguaje de alto nivel orientadoa objetos creado por los desarrolladores del curso conocido como lenguage JACK. 
La implementación a realizar sera la parte del FrontEnd de compilador, es decir, se tomara como entrada un archivo "program.jack" y se retornará un archivo "program.vm" que puede ser ejecutado por el VM Reader que brinda el software generado para el curso.

### Integrantes:
* Geyner Felipe Rojas Torres 
* Isabel Cristina Gómez Balvin
* Iván Daniel Maestre Muza 
* Iván David Ortiz Pineda 

## Parte 1: Definiendo un Token


In [89]:
class Token:
    
    def __init__(self, valor, tipo):
        self.valor = valor
        if tipo in ["keyword", "symbol", "identifier", "integerConstant", "stringConstant"]:
            self.tipo = tipo
        else:
            raise NameError(f"ERROR: No se puede ingresar un token con tipo {tipo}")
        
    
    def __str__(self):
        if self.valor == "<":
            return f"<{self.tipo}> &lt; </{self.tipo}>\n"
        elif self.valor == ">":
            return f"<{self.tipo}> &gt; </{self.tipo}>\n"
        elif self.valor == "&":
            return f"<{self.tipo}> &amp; </{self.tipo}>\n"
        else:
            return f"<{self.tipo}> {self.valor} </{self.tipo}>\n"

    def clase(self):
        return "Token"

In [92]:
tok = Token(",", "symbol")
print(Token(">", "symbol"))
print(Token(";", "symbol"))

<symbol> &gt; </symbol>

<symbol> ; </symbol>



## Parte 2: Creando el Tokenizer

In [93]:
class Tokenizer:
    
    def __init__(self, ruta):
        self.cargar(ruta)
    
    def __str__(self):
        texto = ""
        for token in self._tokens:
            texto = texto + str(token)
        return texto
    
    def cargar(self, ruta):
        self._varciar()
        self._ruta = ruta
        archivo = open(ruta, 'r')
        for linea in archivo.readlines():
            linea = linea.strip()
            if (linea != '') and (linea[0:2] != "//"):
                if linea[0:3] == "/**":
                    pass
                elif linea[0] == "*":
                    if linea.find("*/"):
                        pass
                elif linea.find('//') != -1:
                    self._programa = self._programa + linea[:linea.find('//')].strip()
                else:
                    self._programa = self._programa + linea.strip()
        archivo.close()
        print("Archivo cargado completamente.")
    
    def separar(self):
        sep_comillas = self._programa.split('"')
        #palabras = [''.join([f" {c} " if self.symbol(c) else c for c in val]).split() if ind%2 == 0 else [f'"{val}"'] for ind, val in enumerate(sep_comillas)]
        for ind, var in enumerate(sep_comillas):
            if ind%2 == 0:
                espaciado = []
                for c in var:
                    if self.symbol(c):
                        espaciado.append(f" {c} ")
                    else:
                        espaciado.append(c)
                self._clasificar(''.join(espaciado).split())
            else:
                self._clasificar([f'"{var}"'])
    
    def _clasificar(self, lista):
        for token in lista:
            if self.keyword(token):
                self._tokens.append(Token(token, "keyword"))
            elif self.symbol(token):
                self._tokens.append(Token(token, "symbol"))
            elif '"' in token:
                self._tokens.append(Token(token[1:-1], "stringConstant"))
            elif token.isdigit():
                self._tokens.append(Token(token, "integerConstant"))
            elif not token[0].isdigit():
                self._tokens.append(Token(token, "identifier"))
            else:
                raise NameError(f"ERROR: Declaración de la variable {token} no fue realizada correctamente.")
    
    def escribir(self):
        archivo = open(self._ruta.replace(".jack", "T_.xml"), 'w')
        archivo.write(str(self))
        archivo.close()
        self._varciar()
    
    def _varciar(self):
        self._programa = ""
        self._tokens = []
        
    def get_programa(self):
        return self._programa
    
    def get_lista_tokens(self):
        return self._tokens
    
    def keyword(self, texto):
        return texto in ["class", "constructor", "function", "method", "field", "static", "var", "int", "char", "boolean", "void", "true", "false", "null", "this", "let", "do", "if", "else", "while", "return"]
    
    def symbol(self, texto):
        return texto in ["{", "}", "(", ")", "[", "]", ".", ",", ";", "+", "-", "*", "/", "&", "|", "<", ">", "=", "~"]

In [94]:
tok = Tokenizer(r'C:\Users\idop7\Desktop\nand2tetris\projects\10\ArrayTest\Main.jack')
#print(tok)
tok.separar()
tok.escribir()
print(tok)

Archivo cargado completamente.



## Parte 3: Creando el parser

In [95]:
class Gramatica:
    
    def __init__(self,nombre):
        self.valor = nombre
        self._reglas = []
    
    def __str__(self):
        texto = f"<{self.valor}>\n"
        for regla in self._reglas:
            if regla.clase() == "Gramatica":
                txt = str(regla).split('\n')[:-1]
                txt = ["\t"+t+"\n" for t in txt]
                texto = texto + ''.join(txt)
            else:
                texto = texto + '\t' + str(regla)
        return texto + f"</{self.valor}>\n"
    
    def sum(self, regla):
        self._reglas.append(regla)

    def get_nombre(self):
        return self._nombre
        
    def get_reglas(self):
        return self._reglas
    
    def get_sub(self):
        return self._reglas
    
    def clase(self):
        return "Gramatica"

In [96]:
gram = Gramatica('clase')
gram.sum(Token('class', 'keyword'))
gram.sum(Token('main', 'identifier'))
gram2 = Gramatica('registro')
gram2.sum(gram)
gram2.sum(Token(';', 'symbol'))
print(gram2)

<registro>
	<clase>
		<keyword> class </keyword>
		<identifier> main </identifier>
	</clase>
	<symbol> ; </symbol>
</registro>



In [99]:
class Analizer:
    
    def __init__(self,ruta):
        self._ruta = ruta
        tokenizer = Tokenizer(ruta)
        tokenizer.separar()
        self._tokens = tokenizer.get_lista_tokens()
        self._pos = 0
        self._gramaticas = None
    
    def __str__(self):
        return str(self._gramaticas)
        
    def analizar(self):
        self._gramaticas = self.compila_clase()
        if not self._gramaticas:
            raise NameError(f"ERROR: Hay un error de sintaxis. Por favor revise su código")
    
    def escribir(self):
        archivo = open(self._ruta.replace(".jack", "_.xml"), 'w')
        archivo.write(str(self))
        archivo.close()
    
    def compila_clase(self):
        ini = self._pos
        gram = Gramatica("class")
        token = self._sig()
        if token.valor == "class":
            gram.sum(token)
            token = self._sig()
            if token.tipo == "identifier":
                gram.sum(token)
                token = self._sig()
                if token.valor == "{":
                    gram.sum(token)
                    varc = self.compila_variables_clase()
                    while varc:
                        gram.sum(varc)
                        varc = self.compila_variables_clase()
                    dsr = self.compila_declaracion_subrutina()
                    while dsr:
                        gram.sum(dsr)
                        dsr = self.compila_declaracion_subrutina()
                    token = self._sig()
                    if token.valor == "}":
                        gram.sum(token)
                        return gram
        self._repos(ini)
        return None
    
    def compila_variables_clase(self):
        ini = self._pos
        gram = Gramatica("classVarDec")
        token = self._sig()
        if token.valor in ['static', 'field']:
            gram.sum(token)
            token = self._sig()
            if self.tipo(token):
                gram.sum(token)
                token = self._sig()
                if token.tipo == "identifier":
                    gram.sum(token)
                    token = self._sig()
                    while token.valor == ",":
                        tok_aux = self._sig()
                        if tok_aux.tipo == "identifier":
                            gram.sum(token)
                            gram.sum(tok_aux)
                            token = self._sig()
                        else:
                            self._dev()
                            self._dev()
                            return None
                    if token.valor == ";":
                        gram.sum(token)
                        return gram
        self._repos(ini)
        return None
                        
    def compila_declaracion_subrutina(self):
        ini = self._pos
        gram = Gramatica("subroutineDec")
        token = self._sig()
        if token.valor in ['constructor', 'function', 'method']:
            gram.sum(token)
            token = self._sig()
            if self.tipo(token) or token.valor == "void":
                gram.sum(token)
                token = self._sig()
                if token.tipo == "identifier":
                    gram.sum(token)
                    token = self._sig()
                    if token.valor == "(":
                        gram.sum(token)
                        gram.sum(self.compila_parametros())
                        token = self._sig()
                        if token.valor == ")":
                            gram.sum(token)
                            csr = self.compila_cuerpo_subrutina()
                            if csr:
                                gram.sum(csr)
                                return gram
        self._repos(ini)
        return None;  
    
    def compila_parametros(self):
        ini = self._pos
        gram = Gramatica("parameterList")
        token1 = self._sig()
        token2 = self._sig()
        if self.parametro(token1, token2):
            gram.sum(token1)
            gram.sum(token2)
            token = self._sig()
            while token.valor == ",":
                token1 = self._sig()
                token2 = self._sig()
                if self.parametro(token1, token2):
                    gram.sum(token)
                    gram.sum(token1)
                    gram.sum(token2)
                    token = self._sig()
                else:
                    self._dev()
                    self._dev()
                    self._dev()
                    return gram
            self._dev()
            return gram
        self._repos(ini)
        return gram  
    
    def compila_cuerpo_subrutina(self):
        ini = self._pos
        gram = Gramatica("subroutineBody")
        token = self._sig()
        if token.valor == "{":
            gram.sum(token)
            var = self.compila_variable()
            while var:
                gram.sum(var)
                var = self.compila_variable()
            gram.sum(self.compila_declaraciones())
            token = self._sig()
            if token.valor == "}":
                gram.sum(token)
                return gram
        self._repos(ini)
        return None       
    
    def compila_variable(self):
        ini = self._pos
        gram = Gramatica("varDec")
        token = self._sig()
        if token.valor == "var":
            gram.sum(token)
            token = self._sig()
            if self.tipo(token):
                gram.sum(token)
                token = self._sig()
                if token.tipo == "identifier":
                    gram.sum(token)
                    token = self._sig()
                    while token.valor == ",":
                        tok_aux = self._sig()
                        if tok_aux.tipo == "identifier":
                            gram.sum(token)
                            gram.sum(tok_aux)
                            token = self._sig()
                        else:
                            self._dev()
                            self._dev()
                            return None
                    if token.valor == ";":
                        gram.sum(token)
                        return gram
        self._repos(ini)
        return None
                    
    
    def compila_declaraciones(self):
        ini = self._pos
        gram = Gramatica("statements")
        dec = self.compila_declaracion()
        while dec:
            gram.sum(dec)
            dec = self.compila_declaracion()
        return gram
    
    def compila_declaracion(self):
        token = self._sig()
        self._dev()
        if token.valor == "let":
            return self.compila_let()
        elif token.valor == "if":
            return self.compila_if()
        elif token.valor == "while":
            return self.compila_while()
        elif token.valor == "do":
            return self.compila_do()
        elif token.valor == "return":
            return self.compila_return()
        else:
            return None
    
    def compila_let(self):
        ini = self._pos
        gram = Gramatica("letStatement")
        token = self._sig()
        if token.valor == "let":
            gram.sum(token)
            token = self._sig()
            if token.tipo == "identifier": # nombre de variable
                gram.sum(token)
                token = self._sig()
                if token.valor == "=":
                    gram.sum(token)
                    exp = self.compila_expresion()
                    if exp:
                        gram.sum(exp)
                        token = self._sig()
                        if token.valor == ";":
                            gram.sum(token)
                            return gram
                elif token.valor == "[":
                    gram.sum(token)
                    exp = self.compila_expresion()
                    if exp:
                        gram.sum(exp)
                        token = self._sig()
                        if token.valor == "]":
                            gram.sum(token)
                            token = self._sig()
                            if token.valor == "=":
                                gram.sum(token)
                                exp = self.compila_expresion()
                                if exp:
                                    gram.sum(exp)
                                    token = self._sig()
                                    if token.valor == ";":
                                        gram.sum(token)
                                        return gram
        self._repos(ini)
        return None
                
    def compila_if(self):
        ini = self._pos
        gram = Gramatica("ifStatement")
        token = self._sig()
        if token.valor == "if":
            gram.sum(token)
            token = self._sig()
            if token.valor == "(":
                gram.sum(token)
                exp = self.compila_expresion()
                if exp:
                    gram.sum(exp)
                    token = self._sig()
                    if token.valor == ")":
                        gram.sum(token)
                        token = self._sig()
                        if token.valor == "{":
                            gram.sum(token)
                            stm = self.compila_declaraciones()
                            if stm:
                                gram.sum(stm)
                                token = self._sig()
                                if token.valor == "}":
                                    gram.sum(token)
                                    token = self._sig()
                                    if token.valor != "else":
                                        self._dev()
                                        return gram
                                    else:
                                        gram.sum(token)
                                        token = self._sig()
                                        if token.valor == "{":
                                            gram.sum(token)
                                            stm = self.compila_declaraciones()
                                            if stm:
                                                gram.sum(stm)
                                                token = self._sig()
                                                if token.valor == "}":
                                                    gram.sum(token)
                                                    return gram
        self._repos(ini)
        return None
    
    def compila_while(self):
        ini = self._pos
        gram = Gramatica("whileStatement")
        token = self._sig()
        if token.valor == "while":
            gram.sum(token)
            token = self._sig()
            if token.valor == "(":
                gram.sum(token)
                exp = self.compila_expresion()
                if exp:
                    gram.sum(exp)
                    token = self._sig()
                    if token.valor == ")":
                        gram.sum(token)
                        token = self._sig()
                        if token.valor == "{":
                            gram.sum(token)
                            stm = self.compila_declaraciones()
                            if stm:
                                gram.sum(stm)
                                token = self._sig()
                                if token.valor == "}":
                                    gram.sum(token)
                                    return gram
        self._repos(ini)
        return None
    
    def compila_do(self):
        ini = self._pos
        gram = Gramatica("doStatement")
        token = self._sig()
        if token.valor == "do":
            gram.sum(token)
            token = self._sig()
            if token.tipo == "identifier":
                gram.sum(token)
                token = self._sig()
                if token.valor == "(":
                    gram.sum(token)
                    exl = self.compila_lista_expresion()
                    if exl:
                        gram.sum(exl)
                        token = self._sig()
                        if token.valor == ")":
                            gram.sum(token)
                            token = self._sig()
                            if token.valor == ";":
                                gram.sum(token)
                                return gram
                elif token.valor == ".":
                    gram.sum(token)
                    token = self._sig()
                    if token.tipo == "identifier": # nombre subrutina
                        gram.sum(token)
                        token = self._sig()
                        if token.valor == "(":
                            gram.sum(token)
                            exl = self.compila_lista_expresion()
                            if exl:
                                gram.sum(exl)
                                token = self._sig()
                                if token.valor == ")":
                                    gram.sum(token)
                                    token = self._sig()
                                    if token.valor == ";":
                                        gram.sum(token)
                                        return gram
        self._repos(ini)
        return None
    
    def compila_return(self):
        ini = self._pos
        gram = Gramatica("returnStatement")
        token = self._sig()
        if token.valor == "return":
            gram.sum(token)
            exp = self.compila_expresion()
            if exp:
                gram.sum(exp)
            token = self._sig()
            if token.valor == ";":
                gram.sum(token)
                return gram
        self._repos(ini)
        return None
    
    def compila_lista_expresion(self):
        ini = self._pos
        gram = Gramatica("expressionList")
        exp = self.compila_expresion()
        if exp:
            gram.sum(exp)
            token = self._sig()
            while token.valor == ",":
                exp = self.compila_expresion()
                if exp:
                    gram.sum(token)
                    gram.sum(exp)
                    token = self._sig()
                else:
                    self._dev()
                    return gram
            self._dev()
            return gram
        return gram
    
    def compila_expresion(self):
        ini = self._pos
        gram = Gramatica("expression")
        ter = self.compila_termino()
        if ter:
            gram.sum(ter)
            token = self._sig()
            while self.op(token):
                ter = self.compila_termino()
                if ter:
                    gram.sum(token)
                    gram.sum(ter)
                    token = self._sig()
                else:
                    self._dev()
                    return gram
            self._dev()
            return gram
        self._repos(ini)
        return None
    
    def compila_termino(self):
        ini = self._pos
        gram = Gramatica("term")
        token = self._sig()
        if token.tipo == "integerConstant":
            gram.sum(token)
            return gram
        elif token.tipo == "stringConstant":
            gram.sum(token)
            return gram
        elif self.keywordConstant(token):
            gram.sum(token)
            return gram
        elif token.valor == "(":
            gram.sum(token)
            exp = self.compila_expresion()
            if exp:
                gram.sum(exp)
                token = self._sig()
                if token.valor == ")":
                    gram.sum(token)
                    return gram
        elif self.unaryOp(token):
            gram.sum(token)
            ter = self.compila_termino()
            if ter:
                gram.sum(ter)
                return gram
        elif token.tipo == "identifier": #nombre Subrutina, clase, variable, variable(arreglo)
            gram.sum(token)
            token = self._sig()
            if token.valor == "[":
                gram.sum(token)
                exp = self.compila_expresion()
                if exp:
                    gram.sum(exp)
                    token = self._sig()
                    if token.valor == "]":
                        gram.sum(token)
                        return gram
            elif token.valor == "(":
                gram.sum(token)
                exl = self.compila_lista_expresion()
                if exl:
                    gram.sum(exl)
                    token = self._sig()
                    if token.valor == ")":
                        gram.sum(token)
                        return gram
            elif token.valor == ".":
                gram.sum(token)
                token = self._sig()
                if token.tipo == "identifier": # nombre subrutina
                    gram.sum(token)
                    token = self._sig()
                    if token.valor == "(":
                        gram.sum(token)
                        exl = self.compila_lista_expresion()
                        if exl:
                            gram.sum(exl)
                            token = self._sig()
                            if token.valor == ")":
                                gram.sum(token)
                                return gram
            else:
                self._dev()
                return gram
        self._repos(ini)
        return None
    
    def _sig(self):
        self._pos = self._pos + 1
        return self._tokens[self._pos-1]
    
    def _dev(self):
        self._pos = self._pos - 1
    
    def _repos(self, pos):
        self._pos = pos
    
    def keywordConstant(self, token):
        return token.valor in ["true", "false", "null", "this"]
    
    def op(self, token):
        return token.valor in ["+", "-", "*", "/", "&", "|", "<", ">", "="]
    
    def unaryOp(self, token):
        return token.valor in ["-", "~"]
    
    def tipo(self, token):
        return token.valor in ['int', 'char', 'boolean'] or token.tipo == "identifier"
    
    def parametro(self, token1, token2):
        return self.tipo(token1) and token2.tipo == "identifier"

In [100]:
an = Analizer(r'C:\Users\idop7\Desktop\nand2tetris\projects\10\ArrayTest\Main.jack')
an.analizar()
an.escribir()
print(an)

Archivo cargado completamente.
<class>
	<keyword> class </keyword>
	<identifier> Main </identifier>
	<symbol> { </symbol>
	<subroutineDec>
		<keyword> function </keyword>
		<keyword> void </keyword>
		<identifier> main </identifier>
		<symbol> ( </symbol>
		<parameterList>
		</parameterList>
		<symbol> ) </symbol>
		<subroutineBody>
			<symbol> { </symbol>
			<varDec>
				<keyword> var </keyword>
				<identifier> Array </identifier>
				<identifier> a </identifier>
				<symbol> ; </symbol>
			</varDec>
			<varDec>
				<keyword> var </keyword>
				<keyword> int </keyword>
				<identifier> length </identifier>
				<symbol> ; </symbol>
			</varDec>
			<varDec>
				<keyword> var </keyword>
				<keyword> int </keyword>
				<identifier> i </identifier>
				<symbol> , </symbol>
				<identifier> sum </identifier>
				<symbol> ; </symbol>
			</varDec>
			<statements>
				<letStatement>
					<keyword> let </keyword>
					<identifier> length </identifier>
					<symbol> = </symbol>
					<expressio

## Parte 4: Tabla de Simbolos

In [101]:
class TokenVar:
    
    def __init__(self, token, cat, tipo=None, indice=None, rol=None):
        self.valor = token.valor
        self.tipo = token.tipo
        if cat == "field":
            self.cat = "this"
        else:
            self.cat = cat
        if cat not in ['class', 'subroutine']:
            self.tipo_dato = tipo
            self.ind = indice
            self.rol = rol

    def __str__(self):
        if self.cat == "this":
            cat = "field"
        else:
            cat = self.cat
        if cat not in ['class', 'subroutine']:
            return f"<{self.tipo}> {self.valor} </{self.tipo}> {[cat, self.tipo_dato, self.ind, self.rol]}\n"
        else:
            return f"<{self.tipo}> {self.valor} </{self.tipo}> {[cat]}\n"
    
    def clase(self):
        return "tokenVar"

In [102]:
tok = Token('main', 'identifier')
var = TokenVar(tok, 'variable', 'no', 3, 'def')
var2 = TokenVar(tok, 'class')
print(var)
print(var2)

<identifier> main </identifier> ['variable', 'no', 3, 'def']

<identifier> main </identifier> ['class']



In [103]:
class SymbolTable:
    
    def __init__(self):
        self._tabla = [{"field": [], "static": []}]
    
    def __str__(self):
        txt = ''
        for dic in self._tabla:
            for key, value in dic.items():
                for ind, row in enumerate(value):
                    txt = txt + str(row+[key, ind]) + "\n"
        return txt
    
    def sub(self):
        self._tabla.append({"argument": [], "local": []})
    
    def borraSub(self):
        self._tabla.pop()
    
    def add(self, token, tipo, clase):
        if clase in ["field", "static"]:
            self._tabla[0][clase].append([token.valor, tipo])
        else:
            self._tabla[-1][clase].append([token.valor, tipo])
    
    def trae(self, token, clase):
        if clase in ["field", "static"]:
            tabla = self._tabla[0][clase]
        else:
            tabla = self._tabla[-1][clase]
            
        for ind, reg in enumerate(tabla):
            if reg[0] == token.valor:
                return TokenVar(token, clase, reg[1], ind, 'def')
    
    def busca(self, token):
        tabla = dict(self._tabla[0], **self._tabla[-1])
        for clase, value in tabla.items():
            for ind, reg in enumerate(value):
                if reg[0] == token.valor:
                    return TokenVar(token, clase, reg[1], ind, 'use')

In [104]:
st = SymbolTable()
st.subrutina()
st.add('x', 'int', 'field')
st.add('y', 'int', 'field')
st.add('pointcount', 'point', 'static')
print(st)
st.subrutina()
st.add('this', 'point', 'argument')
st.add('other', 'point', 'argument')
st.add('dx', 'int', 'local')
st.add('dy', 'int', 'local')
print(st)

print(st.busca(Token('dy', 'identifier'), 'use'))
print(st.busca(Token('other', 'identifier'), 'use'))
print(st.busca(Token('pointcount', 'identifier'), 'use'))
print(st.busca(Token('x', 'identifier'), 'use'))

st.subrutina()
st.add('a', 'point', 'argument')
st.add('b', 'point', 'argument')
st.add('sum', 'int', 'local')
print(st)

print(st.busca(Token('a', 'identifier'), 'use'))
print(st.busca(Token('sum', 'identifier'), 'use'))
print(st.busca(Token('pointcount', 'identifier'), 'use'))
print(st.busca(Token('x', 'identifier'), 'use'))
print(st.busca(Token('dx', 'identifier'), 'use'))

AttributeError: 'SymbolTable' object has no attribute 'subrutina'

## Parte 5: Analizador Completo   

In [105]:
class FullAnalizer:
    
    def __init__(self,ruta):
        self._ruta = ruta
        tokenizer = Tokenizer(ruta)
        tokenizer.separar()
        self._tokens = tokenizer.get_lista_tokens()
        self._tabla = SymbolTable()
        self._pos = 0
        self._gramaticas = None
    
    def __str__(self):
        return str(self._gramaticas)
        
    def analizar(self):
        self._gramaticas = self.compila_clase()
        if not self._gramaticas:
            raise NameError(f"ERROR: Hay un error de sintaxis. Por favor revise su código")
    
    def escribir(self):
        archivo = open(self._ruta.replace(".jack", "ST_.xml"), 'w')
        archivo.write(str(self))
        archivo.close()
    
    def compila_clase(self):
        ini = self._pos
        gram = Gramatica("class")
        token = self._sig()
        if token.valor == "class":
            gram.sum(token)
            token = self._sig()
            if token.tipo == "identifier":
                gram.sum(TokenVar(token, 'class'))   # Se especifica que es el nombre de una clase
                token = self._sig()
                if token.valor == "{":
                    gram.sum(token)
                    varc = self.compila_variables_clase()
                    while varc:
                        gram.sum(varc)
                        varc = self.compila_variables_clase()
                    dsr = self.compila_declaracion_subrutina()
                    while dsr:
                        gram.sum(dsr)
                        dsr = self.compila_declaracion_subrutina()
                    token = self._sig()
                    if token.valor == "}":
                        gram.sum(token)
                        return gram
        self._repos(ini)
        return None
    
    def compila_variables_clase(self):
        ini = self._pos
        gram = Gramatica("classVarDec")
        token = self._sig()
        if token.valor in ['static', 'field']:
            cat = token.valor;
            gram.sum(token)
            token = self._sig()
            if self.tipo(token):
                if token.tipo == "identifier":   # Se mira si el tipo declarado es una clase
                    token = TokenVar(token, 'class')
                tipo = token.valor
                gram.sum(token)
                token = self._sig()
                if token.tipo == "identifier":
                    self._tabla.add(token, tipo, cat)   # Se agrega la variable a la tabla
                    token = self._tabla.trae(token, cat)   # Se busca el indice del token en la tabla
                    gram.sum(token)
                    token = self._sig()
                    while token.valor == ",":
                        tok_aux = self._sig()
                        if tok_aux.tipo == "identifier":
                            self._tabla.add(tok_aux, tipo, cat)   # Se agrega la variable a la tabla
                            tok_aux = self._tabla.trae(tok_aux, cat)   # Se busca el indice del token en la tabla
                            gram.sum(token)
                            gram.sum(tok_aux)
                            token = self._sig()
                        else:
                            self._dev()
                            self._dev()
                            return None
                    if token.valor == ";":
                        gram.sum(token)
                        return gram
        self._repos(ini)
        return None
                        
    def compila_declaracion_subrutina(self):
        ini = self._pos
        gram = Gramatica("subroutineDec")
        self._tabla.sub()   # Se añade una nueva subrutina
        token = self._sig()
        if token.valor in ['constructor', 'function', 'method']:
            gram.sum(token)
            if token.valor == 'method':
                self._tabla.add(Token('this', 'keyword'), 'pointer', 'argument')
            token = self._sig()
            if self.tipo(token) or token.valor == "void":
                if token.tipo == "identifier":   # Se mira si el tipo declarado es una clase
                    token = TokenVar(token, 'class')
                gram.sum(token)
                token = self._sig()
                if token.tipo == "identifier":
                    gram.sum(TokenVar(token, 'subroutine'))
                    token = self._sig()
                    if token.valor == "(":
                        gram.sum(token)
                        gram.sum(self.compila_parametros())
                        token = self._sig()
                        if token.valor == ")":
                            gram.sum(token)
                            csr = self.compila_cuerpo_subrutina()
                            if csr:
                                gram.sum(csr)
                                return gram
        self._repos(ini)
        return None;  
    
    def compila_parametros(self):
        ini = self._pos
        gram = Gramatica("parameterList")
        token1 = self._sig()
        token2 = self._sig()
        if self.parametro(token1, token2):
            if token1.tipo == "identifier":   # Se mira si el tipo declarado es una clase
                token1 = TokenVar(token1, 'class')
            tipo = token1.valor
            self._tabla.add(token2, tipo, "argument")   # Se agrega la variable a la tabla
            token2 = self._tabla.trae(token2, "argument")   # Se busca el indice del token en la tabla
            gram.sum(token1)
            gram.sum(token2)
            token = self._sig()
            while token.valor == ",":
                token1 = self._sig()
                token2 = self._sig()
                if self.parametro(token1, token2):
                    if token1.tipo == "identifier":   # Se mira si el tipo declarado es una clase
                        token1 = TokenVar(token1, 'class')
                    tipo = token1.valor
                    self._tabla.add(token2, tipo, "argument")   # Se agrega la variable a la tabla
                    token2 = self._tabla.trae(token2, "argument")   # Se busca el indice del token en la tabla
                    gram.sum(token)
                    gram.sum(token1)
                    gram.sum(token2)
                    token = self._sig()
                else:
                    self._dev()
                    self._dev()
                    self._dev()
                    return gram
            self._dev()
            return gram
        self._repos(ini)
        return gram  
    
    def compila_cuerpo_subrutina(self):
        ini = self._pos
        gram = Gramatica("subroutineBody")
        token = self._sig()
        if token.valor == "{":
            gram.sum(token)
            var = self.compila_variable()
            while var:
                gram.sum(var)
                var = self.compila_variable()
            gram.sum(self.compila_declaraciones())
            token = self._sig()
            if token.valor == "}":
                gram.sum(token)
                return gram
        self._repos(ini)
        return None       
    
    def compila_variable(self):
        ini = self._pos
        gram = Gramatica("varDec")
        token = self._sig()
        if token.valor == "var":
            gram.sum(token)
            token = self._sig()
            if self.tipo(token):
                if token.tipo == "identifier":   # Se mira si el tipo declarado es una clase
                    token = TokenVar(token, 'class')
                tipo = token.valor
                gram.sum(token)
                token = self._sig()
                if token.tipo == "identifier":
                    self._tabla.add(token, tipo, 'local')   # Se agrega la variable a la tabla
                    token = self._tabla.trae(token, 'local')   # Se busca el indice del token en la tabla
                    gram.sum(token)
                    token = self._sig()
                    while token.valor == ",":
                        tok_aux = self._sig()
                        if tok_aux.tipo == "identifier":
                            self._tabla.add(tok_aux, tipo, 'local')   # Se agrega la variable a la tabla
                            tok_aux = self._tabla.trae(tok_aux, 'local')   # Se busca el indice del token en la tabla
                            gram.sum(token)
                            gram.sum(tok_aux)
                            token = self._sig()
                        else:
                            self._dev()
                            self._dev()
                            return None
                    if token.valor == ";":
                        gram.sum(token)
                        return gram
        self._repos(ini)
        return None
                    
    
    def compila_declaraciones(self):
        ini = self._pos
        gram = Gramatica("statements")
        dec = self.compila_declaracion()
        while dec:
            gram.sum(dec)
            dec = self.compila_declaracion()
        return gram
    
    def compila_declaracion(self):
        token = self._sig()
        self._dev()
        if token.valor == "let":
            return self.compila_let()
        elif token.valor == "if":
            return self.compila_if()
        elif token.valor == "while":
            return self.compila_while()
        elif token.valor == "do":
            return self.compila_do()
        elif token.valor == "return":
            return self.compila_return()
        else:
            return None
    
    def compila_let(self):
        ini = self._pos
        gram = Gramatica("letStatement")
        token = self._sig()
        if token.valor == "let":
            gram.sum(token)
            token = self._sig()
            if token.tipo == "identifier":
                token = self._tabla.busca(token)
                gram.sum(token)
                token = self._sig()
                if token.valor == "=":
                    gram.sum(token)
                    exp = self.compila_expresion()
                    if exp:
                        gram.sum(exp)
                        token = self._sig()
                        if token.valor == ";":
                            gram.sum(token)
                            return gram
                elif token.valor == "[":
                    gram.sum(token)
                    exp = self.compila_expresion()
                    if exp:
                        gram.sum(exp)
                        token = self._sig()
                        if token.valor == "]":
                            gram.sum(token)
                            token = self._sig()
                            if token.valor == "=":
                                gram.sum(token)
                                exp = self.compila_expresion()
                                if exp:
                                    gram.sum(exp)
                                    token = self._sig()
                                    if token.valor == ";":
                                        gram.sum(token)
                                        return gram
        self._repos(ini)
        return None
                
    def compila_if(self):
        ini = self._pos
        gram = Gramatica("ifStatement")
        token = self._sig()
        if token.valor == "if":
            gram.sum(token)
            token = self._sig()
            if token.valor == "(":
                gram.sum(token)
                exp = self.compila_expresion()
                if exp:
                    gram.sum(exp)
                    token = self._sig()
                    if token.valor == ")":
                        gram.sum(token)
                        token = self._sig()
                        if token.valor == "{":
                            gram.sum(token)
                            stm = self.compila_declaraciones()
                            if stm:
                                gram.sum(stm)
                                token = self._sig()
                                if token.valor == "}":
                                    gram.sum(token)
                                    token = self._sig()
                                    if token.valor != "else":
                                        self._dev()
                                        return gram
                                    else:
                                        gram.sum(token)
                                        token = self._sig()
                                        if token.valor == "{":
                                            gram.sum(token)
                                            stm = self.compila_declaraciones()
                                            if stm:
                                                gram.sum(stm)
                                                token = self._sig()
                                                if token.valor == "}":
                                                    gram.sum(token)
                                                    return gram
        self._repos(ini)
        return None
    
    def compila_while(self):
        ini = self._pos
        gram = Gramatica("whileStatement")
        token = self._sig()
        if token.valor == "while":
            gram.sum(token)
            token = self._sig()
            if token.valor == "(":
                gram.sum(token)
                exp = self.compila_expresion()
                if exp:
                    gram.sum(exp)
                    token = self._sig()
                    if token.valor == ")":
                        gram.sum(token)
                        token = self._sig()
                        if token.valor == "{":
                            gram.sum(token)
                            stm = self.compila_declaraciones()
                            if stm:
                                gram.sum(stm)
                                token = self._sig()
                                if token.valor == "}":
                                    gram.sum(token)
                                    return gram
        self._repos(ini)
        return None
    
    def compila_do(self):
        ini = self._pos
        gram = Gramatica("doStatement")
        token = self._sig()
        if token.valor == "do":
            gram.sum(token)
            token = self._sig()
            if token.tipo == "identifier":
                ti = token
                token = self._sig()
                if token.valor == "(":
                    gram.sum(TokenVar(ti, 'subroutine'))
                    gram.sum(token)
                    exl = self.compila_lista_expresion()
                    if exl:
                        gram.sum(exl)
                        token = self._sig()
                        if token.valor == ")":
                            gram.sum(token)
                            token = self._sig()
                            if token.valor == ";":
                                gram.sum(token)
                                return gram
                elif token.valor == ".":
                    busqueda = self._tabla.busca(ti)  # mira si es el nombre de una variable o una clase
                    if busqueda:
                        gram.sum(busqueda)
                    else:
                        gram.sum(TokenVar(ti, 'class'))
                    gram.sum(token)
                    token = self._sig()
                    if token.tipo == "identifier":
                        gram.sum(TokenVar(token, 'subroutine'))
                        token = self._sig()
                        if token.valor == "(":
                            gram.sum(token)
                            exl = self.compila_lista_expresion()
                            if exl:
                                gram.sum(exl)
                                token = self._sig()
                                if token.valor == ")":
                                    gram.sum(token)
                                    token = self._sig()
                                    if token.valor == ";":
                                        gram.sum(token)
                                        return gram
        self._repos(ini)
        return None
    
    def compila_return(self):
        ini = self._pos
        gram = Gramatica("returnStatement")
        token = self._sig()
        if token.valor == "return":
            gram.sum(token)
            exp = self.compila_expresion()
            if exp:
                gram.sum(exp)
            token = self._sig()
            if token.valor == ";":
                gram.sum(token)
                return gram
        self._repos(ini)
        return None
    
    def compila_lista_expresion(self):
        ini = self._pos
        gram = Gramatica("expressionList")
        exp = self.compila_expresion()
        if exp:
            gram.sum(exp)
            token = self._sig()
            while token.valor == ",":
                exp = self.compila_expresion()
                if exp:
                    gram.sum(token)
                    gram.sum(exp)
                    token = self._sig()
                else:
                    self._dev()
                    return gram
            self._dev()
            return gram
        return gram
    
    def compila_expresion(self):
        ini = self._pos
        gram = Gramatica("expression")
        ter = self.compila_termino()
        if ter:
            gram.sum(ter)
            token = self._sig()
            while self.op(token):
                ter = self.compila_termino()
                if ter:
                    gram.sum(token)
                    gram.sum(ter)
                    token = self._sig()
                else:
                    self._dev()
                    return gram
            self._dev()
            return gram
        self._repos(ini)
        return None
    
    def compila_termino(self):
        ini = self._pos
        gram = Gramatica("term")
        token = self._sig()
        if token.tipo == "integerConstant":
            gram.sum(token)
            return gram
        elif token.tipo == "stringConstant":
            gram.sum(token)
            return gram
        elif self.keywordConstant(token):
            gram.sum(token)
            return gram
        elif token.valor == "(":
            gram.sum(token)
            exp = self.compila_expresion()
            if exp:
                gram.sum(exp)
                token = self._sig()
                if token.valor == ")":
                    gram.sum(token)
                    return gram
        elif self.unaryOp(token):
            gram.sum(token)
            ter = self.compila_termino()
            if ter:
                gram.sum(ter)
                return gram
        elif token.tipo == "identifier":
            ti = token   # Guardo momentariamente el token
            token = self._sig()
            if token.valor == "[":
                gram.sum(self._tabla.busca(ti))   # Busco el token en la tabla porque es una variable
                gram.sum(token)
                exp = self.compila_expresion()
                if exp:
                    gram.sum(exp)
                    token = self._sig()
                    if token.valor == "]":
                        gram.sum(token)
                        return gram
            elif token.valor == "(":
                gram.sum(TokenVar(ti, 'subroutine'))   # Añado el token como subrutina
                gram.sum(token)
                exl = self.compila_lista_expresion()
                if exl:
                    gram.sum(exl)
                    token = self._sig()
                    if token.valor == ")":
                        gram.sum(token)
                        return gram
            elif token.valor == ".":
                busqueda = self._tabla.busca(ti)  # mira si es el nombre de una variable o una clase
                if busqueda:
                    gram.sum(busqueda)
                else:
                    gram.sum(TokenVar(ti, 'class'))
                #gram.sum(TokenVar(ti, 'class'))   # Añado el token como clase
                gram.sum(token)
                token = self._sig()
                if token.tipo == "identifier":
                    gram.sum(TokenVar(token, 'subroutine'))   # Añado el token como subrutina
                    token = self._sig()
                    if token.valor == "(":
                        gram.sum(token)
                        exl = self.compila_lista_expresion()
                        if exl:
                            gram.sum(exl)
                            token = self._sig()
                            if token.valor == ")":
                                gram.sum(token)
                                return gram
            else:
                gram.sum(self._tabla.busca(ti))   # Busco el token en la tabla porque es una variable
                self._dev()
                return gram
        self._repos(ini)
        return None
    
    def get_reglas(self):
        return self._gramaticas
    
    def _sig(self):
        self._pos = self._pos + 1
        return self._tokens[self._pos-1]
    
    def _dev(self):
        self._pos = self._pos - 1
    
    def _repos(self, pos):
        self._pos = pos
    
    def keywordConstant(self, token):
        return token.valor in ["true", "false", "null", "this"]
    
    def op(self, token):
        return token.valor in ["+", "-", "*", "/", "&", "|", "<", ">", "="]
    
    def unaryOp(self, token):
        return token.valor in ["-", "~"]
    
    def tipo(self, token):
        return token.valor in ['int', 'char', 'boolean'] or token.tipo == "identifier"
    
    def parametro(self, token1, token2):
        return self.tipo(token1) and token2.tipo == "identifier"

In [ ]:
an = FullAnalizer(r'C:\Users\idop7\Desktop\nand2tetris\projects\10\prueba.jack')
an.analizar()
print(an)

In [ ]:
an = FullAnalizer(r'C:\Users\idop7\Desktop\nand2tetris\projects\11\Pong\Main.jack')
an.analizar()
an.
print(an)

## Parte 5: Generador de codigo

In [75]:
class TextGenedator:
    
    def __init__(self, ruta):
        fa = FullAnalizer(ruta)
        fa.analizar()
        self._gramaticas = fa.get_reglas()  # Nodo clase del árbol
        self._vm = []
        self._clase = ''
        self._subrutinas = {}
        self.nif = 0
        self.nwhile = 0
        
    def __str__(self):
        txt = ''
        for ins in self._vm:
            txt = txt + ins + "\n"
        return txt
    
    def generar(self):
        sub = self._gramaticas.get_sub()  # Nodos hijos del nodo clase: 
        self.define_metodos(sub)
        self._clase = sub[1].valor  # Guarda el nombre de la clase
        if len(sub)-4 > 0:           
            for i in range(3, len(sub)-1):
                subdec = sub[i].get_sub()
                if sub[i].valor == "subroutineDec":
                    self._vm.append(f"function {self._clase}.{subdec[2].valor} {self._subrutinas[subdec[2].valor][2]}")
                    #input("Stop")
                    if subdec[0].valor == "method":
                        self._vm.append("push argument 0")
                        self._vm.append("pop pointer 0")
                    if subdec[0].valor == "constructor":
                        self._vm.append(f"push constant {self.cuenta_campos(sub)}")
                        self._vm.append("call Memory.alloc 1")
                        self._vm.append("pop pointer 0")
                    body = subdec[-1].get_sub()   # Entra a los hijos de subroutineBody
                    self.escribe_cuerpo(body[1:-1])   # Solo se para como parametro los Statement y Clases

    def escribir():
        archivo = open(self._ruta.replace(".jack", "_.vm"), 'w')
        archivo.write(prog)
        archivo.close()
        self.vaciar_programa()
                    
    def define_metodos(self, sub):
        if len(sub)-4 > 0:           
            for i in range(3, len(sub)-1):
                subdec = sub[i].get_sub()
                if sub[i].valor == "subroutineDec":
                    subdec = sub[i].get_sub()   # Entra a los nodos hijos del primer subroutineDec
                    nom_subrut = subdec[2].valor
                    fcm_subrut = subdec[0].valor
                    tipo_subrut = subdec[1].valor
                    if fcm_subrut == "method":
                        param = subdec[4].get_sub()   # Enrta a los nodos hijos de parameter list
                        param = len(param)//3 + 1 + (nom_subrut == "method")   # Encuentra el número de parametros a pedir
                    elif fcm_subrut == "function":
                        param = self.cuenta_parametros(subdec[-1].get_sub())
                    else:
                        param = 0
                    self._subrutinas[nom_subrut] = [fcm_subrut, tipo_subrut, param]
    
    def cuenta_parametros(self, body):
        params = 0
        for bdy in body:
            if bdy.valor == "varDec":
                params = params + len(bdy.get_sub()[1:-1])/2
        return int(params)
    
    def cuenta_campos(self, sub):
        campos = 0
        for s in sub:
            if s.valor == "classVarDec":
                campos = campos + len(s.get_sub()[1:-1])/2
        return int(campos)
    
    def escribe_cuerpo(self, body):
        for bdy in body:
            if bdy.valor == "statements":
                stat = bdy.get_sub()
                self.escribe_declaraciones(stat)
    
    def escribe_declaraciones(self, sts):
        ni = self.nif
        nw = self.nwhile
        for dec in sts:
            if dec.valor == "letStatement":
                self.escribe_let(dec.get_sub())
            elif dec.valor == "ifStatement":
                self.nif = self.nif + 1
                self.escribe_if(dec.get_sub(), ni)
            elif dec.valor == "whileStatement":
                self.nwhile = self.nwhile + 1
                self.escribe_while(dec.get_sub(), nw)
            elif dec.valor == "doStatement":
                self.escribe_do(dec.get_sub())
            elif dec.valor == "returnStatement":
                self.escribe_return(dec.get_sub())
    
    def escribe_let(self, let):
        if(len(let) == 8):
            self.escribe_expresion(let[3].get_sub())
            self._vm.append(f"push {let[1].cat} {let[1].ind}")
            self._vm.append("add")
            self.escribe_expresion(let[6].get_sub())
            self._vm.append("pop temp 0")
            self._vm.append("pop pointer 1")
            self._vm.append("push temp 0")
            self._vm.append("pop that 0")
            #self._vm.append("-----------------------")
        else:
            self.escribe_expresion(let[3].get_sub())
            self._vm.append(f"pop {let[1].cat} {let[1].ind}")
    
    def escribe_if(self, ifs, n):
        self.escribe_expresion(ifs[2].get_sub())
        if len(ifs) == 11:
            self._vm.append("if-goto IF-TRUE"+str(n))
            self._vm.append("goto IF-FALSE"+str(n))
            self._vm.append("label IF-TRUE"+str(n))
            self.escribe_declaraciones(ifs[5].get_sub())
            self._vm.append("goto IF-END"+str(n))
            self._vm.append("label IF-FALSE"+str(n))
            self.escribe_declaraciones(ifs[9].get_sub())
            self._vm.append("label IF-END"+str(n))
        else:
            self._vm.append("not")
            self._vm.append("if-goto IF-END"+str(n))
            self.escribe_declaraciones(ifs[5].get_sub())
            self._vm.append("label IF-END"+str(n))
    
    def escribe_while(self, whiles, n):
        self._vm.append("label WHILE-EXP"+str(n))
        self.escribe_expresion(whiles[2].get_sub())
        self._vm.append("not")
        self._vm.append("if-goto WHILE-END"+str(n))
        self.escribe_declaraciones(whiles[5].get_sub())
        self._vm.append("goto WHILE-EXP"+str(n))
        self._vm.append("label WHILE-END"+str(n))
    
    def escribe_do(self, do):
        if do[1].cat == "subroutine":
            par = self._subrutinas[do[1].valor][-1]
            if self._subrutinas[do[1].valor][0] == "method":
                self._vm.append("push pointer 0")
                par = par + 1
            self.escribe_lista_expresiones(do[3].get_sub())
            self._vm.append(f"call {self._clase}.{do[1].valor} {par}")
            self._vm.append("pop temp 0")
        elif do[1].cat == "class":
            self.escribe_lista_expresiones(do[5].get_sub())
            self._vm.append(f"call {do[1].valor}.{do[3].valor} {(len(do[5].get_sub())+1)//2}")
            self._vm.append("pop temp 0")
        else:
            self._vm.append(f"push {do[1].cat} {do[1].ind}")
            self.escribe_lista_expresiones(do[5].get_sub())
            self._vm.append(f"call {do[1].tipo_dato}.{do[3].valor} {(len(do[5].get_sub())+1)//2}")
            self._vm.append("pop temp 0")
    
    def escribe_return(self, returns):
        if len(returns) == 2:
            self._vm.append("push constant 0")
        elif returns[1].valor == 'this':
            self._vm.append("push pointer 0")
        else:
            self.escribe_expresion(returns[1].get_sub())
        self._vm.append("return")

    
    def escribe_lista_expresiones(self, lista):
        if len(lista) > 0:
            self.escribe_expresion(lista[0].get_sub())
            for ind in range(2, len(lista), 2):
                self.escribe_expresion(lista[ind].get_sub())
    
    def escribe_expresion(self, exp):
        self.escribe_termino(exp[0].get_sub())
        for ind in range(2,len(exp),2):
            self.escribe_termino(exp[ind].get_sub())
            self._vm.append(self._op(exp[ind-1].valor))
    
    def escribe_termino(self, term):
        tok1 = term[0]
        if tok1.tipo == "integerConstant":
            self._vm.append(f"push constant {tok1.valor}")
        elif tok1.tipo == "stringConstant":
            self._vm.append(f"push constant {len(tok1.valor)}")
            self._vm.append("call String.new 1")
            for c in tok1.valor:
                self._vm.append(f"push constant {ord(c)}")
                self._vm.append(f"call String.appendChar 2")
        elif tok1.valor == "true":
            self._vm.append("push constant 0")
            self._vm.append("not")
        elif tok1.valor == "false":
            self._vm.append("push constant 0")
        elif tok1.valor == "null":
            self._vm.append("push constant 0")
        elif tok1.valor == "this":
            self._vm.append("push pointer 0")
        elif tok1.valor == "(":
            self.escribe_expresion(term[1].get_sub())
        elif tok1.tipo == "symbol":
            self.escribe_termino(term[1].get_sub())
            self._vm.append(self._un(tok1.valor))
        elif tok1.cat == "subroutine":
            if self._subrutinas[term[0].valor][0] == "method":
                self._vm.append("push pointer 0")
            self.escribe_lista_expresiones(ter[2].get_sub())
            self._vm.append(f"call {self._clase}.{term[0].valor} {self._subrutinas[term[0].valor][-1]}")
            if self._subrutinas[term[0].valor][0] != "method":
                self._vm.append("pop temp 0")
        elif tok1.cat == "class":
            self.escribe_lista_expresiones(term[4].get_sub())
            self._vm.append(f"call {term[0].valor}.{term[2].valor} {(len(term[4].get_sub())+1)//2}")
            #self._vm.append("pop temp 0")
        elif tok1.cat == "pointer" and len(term) == 6:
            self._vm.append(f"push {tok1.cat} {tok1.ind}")
            self.escribe_lista_expresiones(term[4].get_sub())
            self._vm.append(f"call corregir.{term[2].valor} {(len(term[4].get_sub())+1)//2}")
        elif len(term) == 4:
            self.escribe_expresion(term[2].get_sub())
            self._vm.append(f"push {tok1.cat} {tok1.ind}")
            self._vm.append("add")
            self._vm.append("pop pointer 1")
            self._vm.append("push that 0")
        else:
            self._vm.append(f"push {tok1.cat} {tok1.ind}")
            
    def _op(self, operador):
        oper = {'+': "add", '-': "sub", '*': "call Math.multiply 2", '/': "call Math.multiply 2", '&': "and", '|':"or", '<':"lt", '>':"gt", '=':"eq"}
        return oper[operador]
        
    def _un(self, operador):
        ops = {'-': "neg", '~': "not"}
        return ops[operador]

In [76]:
vm = TextGenedator(r"C:\Users\idop7\Desktop\nand2tetris\projects\11\ComplexArrays\Main.jack")
vm.generar()
print(vm)

Archivo cargado completamente.
function Main.main 3
push constant 10
call Array.new 1
pop local 0
push constant 5
call Array.new 1
pop local 1
push constant 1
call Array.new 1
pop local 2
push constant 3
push local 0
add
push constant 2
pop temp 0
pop pointer 1
push temp 0
pop that 0
push constant 4
push local 0
add
push constant 8
pop temp 0
pop pointer 1
push temp 0
pop that 0
push constant 5
push local 0
add
push constant 4
pop temp 0
pop pointer 1
push temp 0
pop that 0
push constant 3
push local 0
add
pop pointer 1
push that 0
push local 1
add
push constant 3
push local 0
add
pop pointer 1
push that 0
push constant 3
add
pop temp 0
pop pointer 1
push temp 0
pop that 0
push constant 3
push local 0
add
pop pointer 1
push that 0
push local 1
add
pop pointer 1
push that 0
push local 0
add
push constant 5
push local 0
add
pop pointer 1
push that 0
push local 0
add
pop pointer 1
push that 0
push constant 7
push constant 3
push local 0
add
pop pointer 1
push that 0
sub
push constant 2
ca

In [46]:
vm = FullAnalizer(r"C:\Users\idop7\Desktop\nand2tetris\projects\11\ComplexArrays\Main.jack")
vm.analizar()
print(vm)

Archivo cargado completamente.
<class>
	<keyword> class </keyword>
	<identifier> Main </identifier> ['class']
	<symbol> { </symbol>
	<subroutineDec>
		<keyword> function </keyword>
		<keyword> void </keyword>
		<identifier> main </identifier> ['subroutine']
		<symbol> ( </symbol>
		<parameterList>
		</parameterList>
		<symbol> ) </symbol>
		<subroutineBody>
			<symbol> { </symbol>
			<varDec>
				<keyword> var </keyword>
				<identifier> Array </identifier> ['class']
				<identifier> a </identifier> ['local', 'Array', 0, 'def']
				<symbol> , </symbol>
				<identifier> b </identifier> ['local', 'Array', 1, 'def']
				<symbol> , </symbol>
				<identifier> c </identifier> ['local', 'Array', 2, 'def']
				<symbol> ; </symbol>
			</varDec>
			<statements>
				<letStatement>
					<keyword> let </keyword>
					<identifier> a </identifier> ['local', 'Array', 0, 'use']
					<symbol> = </symbol>
					<expression>
						<term>
							<identifier> Array </identifier> ['class']
							<symbol> . <

In [ ]:
for i in range(3,3):
    print(i)